In [1]:
%pip install nltk
%pip install tweepy
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import Stream
import pandas as pd
import json
import csv
import sys
import time
import re
import warnings
warnings.filterwarnings('ignore')

In [3]:
ckey = 'o7qidRODRWm5u07PtAWrCpSqf'
csecret = '1Yn60OYMJl5kRCSM0XQ4Lskxh6hjM4VMz51OdsS6dM9DQuQ4MV'
atoken = '1315792402234441730-Dt7AydgVGYI5VN9RRQfnR8Endh0WAc'
asecret = 'jhfiDfB53Z27EqHfvfKbNdo0w2SWLqyyoAkILqgSoaGMK'

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret, 'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])

api = tweepy.API(auth)

In [10]:
def TweetTemizle(Dataset):
    yedek=[]
    print ("Tweetler Temizleniyor...")

    pattern = re.compile('http[s]?://(?:[A-Za-z0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|#[A-Za-z0-9]+|$[A-Za-z0-9]+|@[A-Za-z0-9]+|_[A-Za-z0-9]+|[,.^_$*%-;鶯!?’"“”:=|…]+|\n')
    for i in range(len(Dataset["Tweet_Değiştirilmiş"])):        
        Dataset["Tweet_Değiştirilmiş"][i]=pattern.sub(' ', Dataset["Tweet_Değiştirilmiş"][i])
        
    print ("Tweetler Temizlendi!")
    return Dataset

In [11]:
def toDataFrame(tweets):
    print ("Veriler Kayıt Ediliyor...")
    # COnvert to data frame    
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    DataSet = pd.DataFrame()


    #DataSet['Tweet_Id'] = [tweet.id for tweet in tweets]
    DataSet['Tweet'] = [tweet.text for tweet in tweets]
    DataSet['Tweet_Değiştirilmiş'] = [tweet.text for tweet in tweets]
    #DataSet['Tweet_Tarih'] = [tweet.created_at for tweet in tweets]
    #DataSet['Kullanıcı_Id'] = [tweet.user.id for tweet in tweets]  
    #DataSet['Kullanıcı_Adı'] = [tweet.user.name for tweet in tweets]       
    #DataSet['Dil'] = [tweet.lang for tweet in tweets]
    
    """"
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets] 
    users_retweeted = []    
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    
    DataSet['TweetPlace'] = [i for i in tweets_place]
    DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]
    """
    print ("Veriler Kayıt Edildi!")
    return DataSet

In [12]:
if (not api):
    print ("Bağlantı Yapılamadı!")
    sys.exit(-1)
else:
    print ("Bağlantı Yapıldı!")
    
    aranan=input("Aranacak Kelimeyi Girin: ")
    
    print ("Kelimeye Uygun Tweetler Sorgulanıyor...")
    
    cursor = tweepy.Cursor(api.search_tweets, q=aranan, result_type="new", geocode="55.0000,4.0000,1000km",lang='tr',count=100)
    
    results=[]
    for item in cursor.items(1000): # Remove the limit to 1000
            results.append(item)
            
    print ("Kelimeye Uygun Tweetler Sorgulandı!")
    
    DataSet = toDataFrame(results)    
    TweetTemizle(DataSet)
    DataSet.to_csv('tw.csv',index=False)

Bağlantı Yapıldı!
Kelimeye Uygun Tweetler Sorgulanıyor...
Kelimeye Uygun Tweetler Sorgulandı!
Veriler Kayıt Ediliyor...
Veriler Kayıt Edildi!
Tweetler Temizleniyor...
Tweetler Temizlendi!


In [13]:
Veri=pd.read_csv('tw.csv')

Veri.head()

,Tweet,Tweet_Değiştirilmiş
0,"How do you say “Kahvaltı yaptım, kahvaltı yapıyorum, kahvaltı yapacağım.” in your native language? \n\n-… https://t.co/zTxOl6il1i",How do you say Kahvaltı yaptım kahvaltı yapıyorum kahvaltı yapacağım in your native language
1,Canlı | Monaco&amp;Trabzonspor canlı anlatım izle! Monaco&amp;Trabzonspor maçı saat kaçta hangi kanalda? İlk 11'ler...… https://t.co/pejZ0vxoaR,Canlı Monaco amp Trabzonspor canlı anlatım izle Monaco amp Trabzonspor maçı saat kaçta hangi kanalda İlk ler
2,Sabahın köründe egenin güzel sularınada yaşam koşusu yapacağım. Bok kusun istanbul-ankara.,Sabahın köründe egenin güzel sularınada yaşam koşusu yapacağım Bok kusun istanbul ankara
3,@devbirbozayi istanbul harici = kirsal,istanbul harici kirsal
4,ben örsün kerpetenin mühendisi\nistanbul platosundan\narizona çölüne kadar\nilmek ilmek işledim\nbütün namuskar\nbütün i… https://t.co/oh1djQAN3i,ben örsün kerpetenin mühendisi istanbul platosundan arizona çölüne kadar ilmek ilmek işledim bütün namuskar bütün i


In [14]:
Veri.tail()

,Tweet,Tweet_Değiştirilmiş
485,@mt_goksu Seni istanbul a baskan mi yapacak istanbul’lu hadi ordan.,Seni istanbul a baskan mi yapacak istanbul lu hadi ordan
486,@mertrecep12 @mehmetcantemel @AlcayirCengiz Buralrda imar planını büyük şehir yapar. 2017 yılında bunu Cumhur başka… https://t.co/H0LwNmXrzm,Buralrda imar planını büyük şehir yapar yılında bunu Cumhur başka
487,İstanbul-Taksim Terör Saldırısını Lanetliyoruz! - https://t.co/CcuSF4hyNJ https://t.co/vdGISgaqou,İstanbul Taksim Terör Saldırısını Lanetliyoruz
488,"@jaresal_jakmak @SunExpress Hayır zürihe dönüyorum, haftaya istanbul🌻",Hayır zürihe dönüyorum haftaya istanbul🌻
489,"kar sezonu başlamadı, bodrum soğuk vakit varken tam istanbul’u gezme mevsimi. hem iki üç fotoğraf çekilir. https://t.co/h0oIS6sOT6",kar sezonu başlamadı bodrum soğuk vakit varken tam istanbul u gezme mevsimi hem iki üç fotoğraf çekilir


In [9]:
Veri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Tweet                263 non-null    object
 1   Tweet_Değiştirilmiş  263 non-null    object
dtypes: object(2)
memory usage: 4.2+ KB
